<a href="https://colab.research.google.com/github/Alrix010/hypotez/blob/master/SANDBOX/alexandr/LLM-HOWTO/LLM_%D1%88%D0%B0%D0%B3_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install transformers torch sentencepiece accelerate huggingface_hub  #--quiet

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install accelerate huggingface_hub  #--quiet

In [5]:
import json
import glob
import os
import pandas as pd
from datasets import Dataset
import pandas as pd
os.environ["WANDB_DISABLED"] = "true"

# from huggingface_hub import notebook_login
# notebook_login()

from google.colab import userdata
from huggingface_hub import login

## Загрузка модели и данных

Для текстовой классификации я выбрал модель *distilbert-base-uncased*

#### Модель `"distilbert-base-uncased"`

— облегчённая (дистиллированная) версия BERT. Название по частям:

 **1. `distilbert`**  
- Это **дистиллированная (сжатая) версия BERT**.  
- Обучена методом **knowledge distillation** (перенос знаний из большой модели `bert-base-uncased` в меньшую).  
- Сохраняет ~95% качества оригинального BERT, но **в 2 раза быстрее** и **на 40% меньше** по размеру.

 **2. `base`**  
- Размер модели: **"base"** (12 слоёв, 768 скрытых размерностей, 110 млн параметров).  
- Есть также `tiny`, `mini`, `small` для ещё более лёгких экспериментов.

 **3. `uncased`**  
- Модель **не различает регистр букв** (все тексты приводятся к нижнему регистру перед обработкой).  
- Например: `"Hello"` и `"hello"` будут считаться одинаковыми.  
- Если регистр важен (например, для именованных сущностей), следует изпользовать **`cased`**-версии.

---

#### **Когда её использовать?**  
- Для задач **классификации текста**, **NER**, **вопросо-ответных систем**.  
- Если нужно **экономить ресурсы** (Colab/ноутбук с ограниченным GPU).  
- Для экспериментов перед переходом на большие модели (например, `bert-large`).

---


#### **Альтернативы**  
- **`bert-base-uncased`** — оригинальная BERT (медленнее, но точнее).  
- **`distilroberta-base`** — дистиллированная версия RoBERTa.  
- **`google/electra-small`** — лёгкая модель.  

Если нужна **модель для русского языка**, 👇  
- `"DeepPavlov/rubert-base-cased"`  
- `"cointegrated/rubert-tiny2"` (очень лёгкая).  


### Google Drive и пути

In [6]:
from google.colab import userdata
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:

# Пути
storage:str = '/content/drive/MyDrive/llm'
train_data_path:str = f'{storage}/train_data'
log_path:str = f'{storage}/logs'
results_path:str = f'{storage}/results'
checkpoints_path:str = f'{storage}/checkpoints'

# Список файлов для тренировки
json_train_files_list:list | str = glob.glob(os.path.join(train_data_path, '*.json'))
json_train_files_list:list = json_train_files_list if isinstance(json_train_files_list, list) else [json_train_files_list]



#### CSV

In [ ]:

#df = pd.read_csv('/content/drive/MyDrive/hypo69/llam_20250516T064847-195.csv')
#dataset = Dataset.from_pandas(df)

#train_df = pd.read_csv("train.csv")
#eval_df = pd.read_csv("val.csv")

#train_dataset = Dataset.from_pandas(train_df)
#eval_dataset = Dataset.from_pandas(eval_df)

#### JSON

In [8]:

records:list = []
for file_path in json_train_files_list:
    with open(file_path, 'r', encoding='utf-8') as f:
        try:
            data = json.load(f)

            # Если это список словарей
            if isinstance(data, list):
                records.extend(data)
            # Если это одиночный словарь
            elif isinstance(data, dict):
                records.append(data)

        except Exception as e:
            print(f'Ошибка при загрузке {file_path}: {e}')

labels_dict: dict = {}
try:
    with open(f'{storage}/labels.json', 'r', encoding='utf-8') as f:
        labels_dict = json.load(f)
except Exception as e:
    print

In [9]:
df = pd.DataFrame(records)
# Удаление строк с пустыми значениями в столбце 'text'
df_cleaned = df.dropna(subset=['text'])  # Удаляет None/NaN
df_cleaned = df_cleaned[df_cleaned['text'].astype(str).str.strip() != ""]  # Удаляет пустые строки
print(f"Было строк: {len(df)}, стало: {len(df_cleaned)}")
print("Примеры оставшихся текстов:")
print(df_cleaned['text'].head())


Было строк: 357, стало: 357
Примеры оставшихся текстов:
0    <div id="__nuxt"><div><div><div><ds-header acc...
1    <div id="__nuxt"><div><div><div><ds-header acc...
2    <div id="__nuxt"><div><div><div><ds-header acc...
3    <div class="uwy userway_p1" data-uw-feature-ig...
4    <div id="__nuxt"><div><div><div><ds-header acc...
Name: text, dtype: object


In [10]:
dataset:Dataset = Dataset.from_pandas(df_cleaned)
dataset = dataset.train_test_split(test_size=0.2)  # 80% train, 20% eval
train_dataset:Dataset = dataset["train"]
eval_dataset:Dataset = dataset["test"]

## Процесс обучения

Модель ожидает последовательности фиксированной длины (512 токенов)

Если данные содержат слишком длинные тексты (461 токен вместо ожидаемых 41)

Проблема возникает при попытке создать батчи из разрозненных по длине последовательностей

In [11]:
# -*- coding: utf-8 -*-
"""
Модуль для создания и обучения пайплайна классификации текста.
================================================================
Модуль определяет класс TextClassificationPipeline, который инкапсулирует
логику токенизации, подготовки датасетов и обучения модели.
Зависимости:
    - transformers
    - datasets
    - torch
    - numpy
    - pandas
    - huggingface_hub (для загрузки на Hub)
"""

from typing import Dict, Tuple, Any, Optional
from pathlib import Path # Импортируем Path для работы с путями

from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers.tokenization_utils_base import BatchEncoding
from transformers.trainer_utils import EvalPrediction
from datasets import DatasetDict, Dataset
import torch
import numpy as np
import pandas as pd
import math


class TextClassificationPipeline:
    """
    Пайплайн для классификации текста.
    """
    def __init__(self,
                 model_name: str = 'distilbert-base-uncased',
                 num_labels: int = 40,
                 hub_model_id: Optional[str] = None,
                 checkpoints_path: str = './results',
                 labels_map_dict: Optional[Dict[str, int]] = None):

        """
        Инициализация пайплайна.

        Args:
            model_name (str, optional): Имя или путь к предварительно обученной модели.
            num_labels (int, optional): Количество меток для классификации.
            hub_model_id (Optional[str], optional): ID модели на Hugging Face Hub.
            checkpoints_path (str, optional): Путь к директории для сохранения чекпоинтов.
                                              По умолчанию './results'.
        """
        self.tokenizer: AutoTokenizer
        self.model: AutoModelForSequenceClassification
        self.num_labels: int
        self.hub_model_id: Optional[str] = hub_model_id
        self.checkpoints_path: str = checkpoints_path # Сохраняем путь для чекпоинтов

        try:
            self.tokenizer = AutoTokenizer.from_pretrained(model_name)
            self.model = AutoModelForSequenceClassification.from_pretrained(
                model_name,
                num_labels=num_labels,
                problem_type='single_label_classification'
            )
            self.num_labels = num_labels
        except OSError as ex:
            print(f"Не удалось загрузить модель или токенизатор: {model_name}\nException: {ex}")
            raise
        except Exception as ex:
            print(f"Непредвиденная ошибка при инициализации TextClassificationPipeline: {model_name}\nException: {ex}")
            raise

    def tokenize(self,
                 input_data: Dict[str, Any],
                 max_length: int = 512,
                 stride_ratio: float = 0.5) -> BatchEncoding:
        """
        Токенизация текстовых данных с перекрытием (внахлест).

        Args:
        input_data (Dict[str, Any]): Словарь, содержащий ключ 'text' со списком текстов.
                                    Может также содержать 'labels'.
        max_length (int, optional): Максимальная длина одного чанка. По умолчанию 512.
        stride_ratio (float, optional): Коэффициент для определения шага (stride)
                                        как доля от max_length.
                                        Например, 0.5 означает перекрытие в половину max_length.
                                        По умолчанию 0.5.

        Returns:
        BatchEncoding: Токенизированные данные. Если текст был разделен на чанки,
                        для каждого чанка будет соответствующая запись. Метки дублируются
                        для всех чанков, порожденных одним исходным примером.
        """
        tokenized: BatchEncoding

        actual_stride = min(max(0, int(max_length * stride_ratio)), max_length -1)
        if actual_stride <= 0 and max_length > 0 :
            if stride_ratio > 0 :
                 print(f"Предупреждение: stride_ratio > 0, но max_length ({max_length}) слишком мал. Stride установлен в 1.")
                 actual_stride = 1
            else:
                 actual_stride = 0

        print(f"Токенизация с max_length={max_length}, stride={actual_stride} (stride_ratio={stride_ratio})")

        tokenized = self.tokenizer(
            input_data['text'],
            truncation=True,
            max_length=max_length,
            stride=actual_stride,
            return_overflowing_tokens=True,
            padding='max_length',
            return_tensors='pt'
        )

        if 'labels' in input_data and 'overflow_to_sample_mapping' in tokenized:
            original_labels = input_data['labels']
            new_labels = []
            for i in range(len(tokenized['input_ids'])):
                original_sample_index = tokenized['overflow_to_sample_mapping'][i]
                new_labels.append(original_labels[original_sample_index])
            tokenized['labels'] = new_labels
        elif 'labels' in input_data:
            tokenized['labels'] = input_data['labels']


        if 'overflow_to_sample_mapping' in tokenized:
            del tokenized['overflow_to_sample_mapping']
        # Если токенизатор добавляет 'offset_mapping' и он не нужен модели,
        # его тоже можно удалить здесь, хотя обычно модели его игнорируют, если он не в их сигнатуре.
        # if 'offset_mapping' in tokenized:
        #     del tokenized['offset_mapping']
        # --------------------------

        return tokenized

    def prepare_datasets(self, dataset_dict: DatasetDict) -> Tuple[Dataset, Dataset] | Tuple[None, None]:
        """ Подготовка (токенизация) датасетов. """
        tokenized_datasets: DatasetDict
        error_message: str

        for split_name, current_dataset in dataset_dict.items():
            if 'labels' not in current_dataset.column_names:
                error_message = f"Датасет '{split_name}' не содержит 'labels'. Колонки: {current_dataset.column_names}"
                print(error_message)
                raise ValueError(error_message)
            if 'text' not in current_dataset.column_names:
                error_message = f"Датасет '{split_name}' не содержит 'text'. Колонки: {current_dataset.column_names}"
                print(error_message)
                raise ValueError(error_message)

        try:
            print(f"Начало токенизации для датасетов: {list(dataset_dict.keys())}...")
            tokenized_datasets = dataset_dict.map(
                self.tokenize,
                batched=True,
                remove_columns=['text']
            )
            print("Токенизация датасетов завершена.")
        except Exception as ex:
            print(f"Ошибка во время токенизации: {ex}")
            return None, None

        if 'train' not in tokenized_datasets or 'test' not in tokenized_datasets:
            print(f"Ключи 'train' или 'test' отсутствуют. Ключи: {tokenized_datasets.keys()}")
            return None, None

        return tokenized_datasets['train'], tokenized_datasets['test']

    def train(self, train_dataset: Dataset, eval_dataset: Dataset) -> None:
        """ Запуск процесса обучения модели. """
        training_args: TrainingArguments
        trainer: Trainer

        if not train_dataset:
            print("Ошибка: Обучающий датасет пуст.")
            return

        per_device_batch_size: int = 8
        steps_per_epoch: int = math.ceil(len(train_dataset) / per_device_batch_size)
        if steps_per_epoch == 0:
            steps_per_epoch = 1
        print(f"Рассчитанное количество шагов на эпоху: {steps_per_epoch}")

        args_dict = {
            'output_dir': self.checkpoints_path,
            'per_device_train_batch_size': per_device_batch_size,
            'per_device_eval_batch_size': 8,
            'num_train_epochs': 3,
            'do_eval': True,
            'eval_steps': steps_per_epoch,
            'logging_steps': steps_per_epoch,
            'logging_first_step': True,
            'save_steps': steps_per_epoch,
            'remove_unused_columns': False,
            # 'overwrite_output_dir': True, # Раскомментировать, для перезписи output_dir без ошибок
        }

        if self.hub_model_id:
            args_dict['push_to_hub'] = True
            args_dict['hub_model_id'] = self.hub_model_id
            print(f"Модель будет загружена на Hugging Face Hub как: {self.hub_model_id}")
            # Для управления частотой загрузки на Hub, если push_to_hub=True:
            # args_dict['hub_strategy'] = "every_save" # Загружать при каждом сохранении чекпоинта
            # args_dict['hub_strategy'] = "epoch" # Загружать в конце каждой эпохи (потребует соответствующей save_strategy)
            # args_dict['hub_strategy'] = "end" # Загружать только в самом конце обучения
            # Если hub_strategy не указан, поведение по умолчанию обычно "end" или синхронизировано с save_strategy.
            # Для старых версий, где эти опции могут не работать, поведение может быть только "в конце".
        else:
            print("Модель НЕ будет загружена на Hugging Face Hub (hub_model_id не указан).")

        training_args = TrainingArguments(**args_dict)

        trainer = Trainer(
            model=self.model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=eval_dataset,
            tokenizer=self.tokenizer,
            compute_metrics=self._compute_metrics
        )

        try:
            print(f"Начало обучения... Чекпоинты будут сохраняться в: {self.checkpoints_path}")
            trainer.train()

            # Сохраняем финальную модель локально в отдельную директорию (если нужно)
            # или можно положиться на последний чекпоинт в self.checkpoints_path
            final_model_path = Path(self.checkpoints_path) / "final_model_after_training"
            trainer.save_model(str(final_model_path))
            print(f"Обучение завершено. Финальная модель сохранена локально в {final_model_path}")

            # Ручная загрузка на Hub (если push_to_hub в TrainingArguments не сработал или если хочется сделать это явно)
            if self.hub_model_id and not training_args.push_to_hub:
                print(f"Попытка загрузить модель на Hub вручную: {self.hub_model_id}")
                try:
                    # trainer.push_to_hub() # обычно загружает все, включая токенизатор из args
                    # или более явно:
                    self.tokenizer.push_to_hub(self.hub_model_id, commit_message="Push tokenizer after training")
                    self.model.push_to_hub(self.hub_model_id, commit_message="Push model after training")
                    print(f"Модель и токенизатор должны быть загружены на {self.hub_model_id}")
                except Exception as e_push:
                    print(f"Ошибка при ручной загрузке на Hub: {e_push}")

        except Exception as ex:
            print(f'Ошибка во время обучения или сохранения/загрузки модели: {ex}')
            raise

    def _compute_metrics(self, eval_pred: EvalPrediction) -> Dict[str, float]:
        """ Вычисление метрик.
        Назначение:
          Вычисляет метрики качества модели на основе предсказаний и истинных меток. Эта функция передается в Trainer.
        Действия:
          - Извлекает predictions (логиты) и labels (истинные метки).
          - Применяет np.argmax к логитам вдоль оси классов (axis=1) для получения предсказанных классов (ID с наибольшей вероятностью).
          - Вычисляет точность (accuracy) как долю совпадений между предсказанными и истинными метками.
          - Возвращает словарь с метрикой, например, {'accuracy': 0.95}.
        Args:
          eval_pred (EvalPrediction): Объект, содержащий predictions (логиты модели, т.е. сырые выходы до применения softmax) и label_ids (истинные метки).
        Returns:
          Dict[str, float]: Словарь с метрикой качества.
        """
        predictions_logits: np.ndarray = eval_pred.predictions
        labels: np.ndarray = eval_pred.label_ids
        predictions: np.ndarray = np.argmax(predictions_logits, axis=1)
        accuracy: float | np.float_ = np.mean(predictions == labels)
        return {'accuracy': accuracy}



In [ ]:

if __name__ == '__main__':


    HF_MODEL_ID: Optional[str] = None
    try:
        hf_token = userdata.get('HF_TOKEN')
        if not hf_token:
            print("Токен HF_TOKEN не найден в секретах.")
        else:
            print("Токен HF получен.")
            login(token=hf_token)
            print("Успешная авторизация на HF Hub.")
            HF_MODEL_ID = "Alrix010/my_model_from_existing_datasets" # <--- ИМЯ МОДЕЛИ
    except userdata.SecretNotFoundError:
        print("Секрет HF_TOKEN не найден.")
    except Exception as e:
        print(f"Ошибка при авторизации на HF Hub: {e}")
        HF_MODEL_ID = None


    if 'checkpoints_path' not in locals() and 'checkpoints_path' not in globals():
        checkpoints_path: str = "./my_final_checkpoints"
        print(f"Переменная 'checkpoints_path' не найдена, используется значение по умолчанию: {checkpoints_path}")


    # -------------------------------- labels_dict -----------------
    if labels_dict:
        calculated_num_labels_main = len(labels_dict)
        # Опциональные проверки для labels_dict
        if not all(isinstance(v, int) for v in labels_dict.values()):
            print("Предупреждение: Не все значения в labels_dict являются целыми числами.")
        if not all(isinstance(k, str) for k in labels_dict.keys()):
            print("Предупреждение: Не все ключи в labels_dict являются строками.")
    else:
        labels_dict = {} # Гарантия, что это пустой словарь, а не None
        print("Предупреждение: labels_dict пуст или не был загружен. num_labels будет 0.")

    if calculated_num_labels_main == 0:
        print("Критическая ошибка: Количество меток (num_labels) равно 0.")
        # exit()
    else:
        print(f"Итоговое количество меток (num_labels) для модели: {calculated_num_labels_main}")


    pipeline_instance: Optional[TextClassificationPipeline] = None
    dataset_dict_for_pipeline_main: Optional[DatasetDict] = None
    prepared_tokenized_datasets: Optional[Tuple[Dataset, Dataset] | Tuple[None, None]] = None
    final_tokenized_train_data: Optional[Dataset] = None
    final_tokenized_eval_data: Optional[Dataset] = None

    try:
        # Проверка, существуют ли train_dataset и eval_dataset
        if ('train_dataset' not in locals() and 'train_dataset' not in globals()) or \
           ('eval_dataset' not in locals() and 'eval_dataset' not in globals()):
            raise NameError("Переменные 'train_dataset' и/или 'eval_dataset' не определены. "
                            "Убедитесь, что они созданы в предыдущих ячейках.")

        # Проверка, что они не None (на случай, если они были объявлены, но не инициализированы)
        if train_dataset is None or eval_dataset is None:
             raise ValueError("'train_dataset' или 'eval_dataset' являются None. "
                              "Убедитесь, что они корректно созданы.")

        print(f"Используется существующий train_dataset ({len(train_dataset)} записей) "
              f"и eval_dataset ({len(eval_dataset)} записей).")

        # Проверка наличия колонок 'text' и 'labels' в существующих датасетах
        for ds_name, ds_object in [("train_dataset", train_dataset), ("eval_dataset", eval_dataset)]:
            if ds_object: # Проверка, что объект не None
                if 'text' not in ds_object.column_names or 'labels' not in ds_object.column_names:
                    raise ValueError(f"{ds_name} должен содержать колонки 'text' и 'labels'. "
                                     f"Текущие колонки: {ds_object.column_names}")
                # Проверка, что все метки в данных находятся в диапазоне [0, num_labels-1]
                # Эта проверка важна, так как `num_labels` берется из `labels_dict`
                if calculated_num_labels_main > 0: # Только если у нас есть ожидаемое кол-во меток
                    all_labels_in_ds = set(ds_object['labels'])
                    if all_labels_in_ds and \
                       (max(all_labels_in_ds) >= calculated_num_labels_main or min(all_labels_in_ds) < 0):
                        raise ValueError(
                            f"Метки в {ds_name} (диапазон: {min(all_labels_in_ds)}-{max(all_labels_in_ds)}) "
                            f"выходят за пределы ожидаемого диапазона [0, {calculated_num_labels_main-1}], "
                            f"определенного из actual_labels_dict. Уникальные метки: {sorted(list(all_labels_in_ds))}"
                        )
                elif calculated_num_labels_main == 0 and ds_object['labels']: # Если actual_labels_dict пуст, но в данных есть метки
                    print(f"Предупреждение: actual_labels_dict пуст, но {ds_name} содержит метки. num_labels будет 0, что вызовет ошибку.")


        if calculated_num_labels_main > 0: # Продолжаем только если num_labels определен
            dataset_dict_for_pipeline_main = DatasetDict({
                'train': train_dataset,
                'test': eval_dataset
            })

            pipeline_instance = TextClassificationPipeline(
                num_labels=calculated_num_labels_main,
                hub_model_id=HF_MODEL_ID,
                checkpoints_path=checkpoints_path,
                labels_map_dict= labels_dict
            )
        else:
            print("Обучение не будет запущено, так как calculated_num_labels_main равен 0 (проблема с actual_labels_dict).")

        if pipeline_instance and dataset_dict_for_pipeline_main:
            prepared_tokenized_datasets = pipeline_instance.prepare_datasets(dataset_dict_for_pipeline_main)

            if prepared_tokenized_datasets and prepared_tokenized_datasets[0] and prepared_tokenized_datasets[1]:
                final_tokenized_train_data, final_tokenized_eval_data = prepared_tokenized_datasets
                print('Данные успешно токенизированы:')
                print(f'Train: {len(final_tokenized_train_data)} п., Колонки: {final_tokenized_train_data.column_names}')
                print(f'Eval: {len(final_tokenized_eval_data)} п., Колонки: {final_tokenized_eval_data.column_names}')

                pipeline_instance.train(final_tokenized_train_data, final_tokenized_eval_data)
            else:
                print('Ошибка токенизации. Обучение не будет запущено.')
        elif not pipeline_instance and calculated_num_labels_main > 0:
             print("Пайплайн не был инициализирован.")


    except NameError as ne: # Если train_dataset или eval_dataset не существуют
        print(f"Ошибка NameError: {ne}")
        print("Убедитесь, что переменные train_dataset, eval_dataset и labels_dict (если не загружается здесь) определены в предыдущих ячейках.")
    except ValueError as ex:
        print(f'Критическая ошибка (ValueError): {str(ex)}')
    except Exception as ex:
        import traceback
        print(f'Непредвиденная ошибка: {str(ex)}')
        print("Трассировка:")
        traceback.print_exc()

Токен HF получен.
Успешная авторизация на HF Hub.
Итоговое количество меток (num_labels) для модели: 37
Используется существующий train_dataset (285 записей) и eval_dataset (72 записей).


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Начало токенизации для датасетов: ['train', 'test']...


Map:   0%|          | 0/285 [00:00<?, ? examples/s]

Токенизация с max_length=512, stride=256 (stride_ratio=0.5)


...

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Токенизация датасетов завершена.
Данные успешно токенизированы:
Train: 8560 п., Колонки: ['labels', 'input_ids', 'attention_mask']
Eval: 1818 п., Колонки: ['labels', 'input_ids', 'attention_mask']
Рассчитанное количество шагов на эпоху: 1070
Модель будет загружена на Hugging Face Hub как: Alrix010/my_model_from_existing_datasets


<ipython-input-11-b7f8b28ace21>:212: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Начало обучения... Чекпоинты будут сохраняться в: /content/drive/MyDrive/llm/checkpoints


Step,Training Loss
1,3.558200
